# Dataset Creation

## Aim
Create a dataset for **future analysis**.

---

## Necessary Downloads
Before starting, be sure to have downloaded the following material.

### 1. MSigDB Gene Sets (Human)
Download the entire MSigDB for **Human** in **JSON** format (`Human Gene Set JSON file set (ZIPped)`).
- **Link**: [https://www.gsea-msigdb.org/gsea/downloads.jsp](https://www.gsea-msigdb.org/gsea/downloads.jsp)

### 2. UniProt Human Proteome
Download the list of all **HUMAN proteins** (UniProt human proteome - **UP000005640**).
- **Link (Website)**: [https://www.uniprot.org/proteomes/UP000005640](https://www.uniprot.org/proteomes/UP000005640)
- **Programmatic Download (Terminal)**:

```bash
wget -O human_proteome.tsv.gz "[https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession,reviewed,id,protein_name,gene_names,organism_name,sequence&format=tsv&query=(proteome:UP000005640](https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession,reviewed,id,protein_name,gene_names,organism_name,sequence&format=tsv&query=(proteome:UP000005640))"
gunzip human_proteome.tsv.gz
```

<!-- ### 3. UniProt Gene-Protein mapping

[UniProt protein-gene Mapping](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/by_organism/HUMAN_9606_idmapping.dat.gz)


### 3. UniRef50 Clusters

#### A. Initial Programmatic Download (Partial)
This method retrieves all clusters with at least one human protein, but is a **subsample** of the full UniRef50.
- **Link (Website)**: [https://www.uniprot.org/uniref?query=%28identity%3A0.5%29+AND+%28taxonomy_id%3A9606%29](https://www.uniprot.org/uniref?query=%28identity%3A0.5%29+AND+%28taxonomy_id%3A9606%29)
- **Programmatic Download (Terminal)**:
    - **ATTENTION**: This call retrieves all clusters with at least 1 human protein, *not* just the human proteins themselves.

```bash
curl -o uniref50_human.tsv.gz "[https://rest.uniprot.org/uniref/stream?compressed=true&fields=id,name,organism,length,identity,count,members&format=tsv&query=((identity:0.5)+AND+(taxonomy_id:9606](https://rest.uniprot.org/uniref/stream?compressed=true&fields=id,name,organism,length,identity,count,members&format=tsv&query=((identity:0.5)+AND+(taxonomy_id:9606)))"
gunzip uniref50_human.tsv.gz
``` -->

<!-- ### 5. BioMart Gene-Protein mapping

Download BioMart gne-protein mapping, besure to include **Gene Name** and **Transcript Name**.
[link](https://www.ensembl.org/biomart/martview/3aaaf734b93facdfad8207234204cc31) -->


# Differences

- OK: no "PROLIFERATION"| CDK5 |CANCER| APOPTOSIS-- much stringet find
- OK: do not care about crucial for mitosis, but not for controlling whether or when mitosis happens. (ex spidnle) They are executors, not decision-makers.
    - recome cell cycle is too broad!!!
    - My concern is that including genes encoding structural machinery might introduce too much variability for the model to learn effectively.
- OK: not use as negative proteisn in the same lcuster as postive
    - Cluster UniRef50_Q5VXH4 contains proteins [Q5VXH4, A3QJZ7, Q5VWM4]
    - Say 2 are labeled positive, 1 is labeled negative
    - ut this isn't learning cell-cycle function — it's learning "this cluster tends to be cell-cycle"
    - his creates an impossible learning problem. The model can't possibly learn that these should have different labels because there's almost no signal in the embeddings to distinguish them.
- change egative class:
    - Negative = "protein annotated to genes known to be involved in non-cell-cycle pathways"
        - For example: neuronal proteins, muscle-specific proteins, proteins involved in cell death/apoptosis, structural proteins
        - "neuronal development," "muscle differentiation," "extracellular matrix"
    - or housekeep genes
- OK: do not sample psotve genes 8take all)
- OK: aplit by  cluster in train/test -- no dataleackage


CONCENRS
- false negative: : Just because a gene wasn't in your MSigDB search doesn't mean the protein isn't involved in cell-cycle.
- Genes that appear in ambiguous frequency aren't necessarily non-cell-cycle



# Hyperparameters

In [2]:
import json
import os
import re
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.model_selection import train_test_split
import random
import torch

import utils.dataset_functions as dataf

# Initializations
SEED=42
random.seed(SEED)
np.random.seed(SEED)

# Directory containing MSigDB JSON files
JSON_DIR = "/home/gdallagl/myworkdir/data/MSigDB/msigdb_v2025.1.Hs_json_files_to_download_locally"

# cellcycel geensets saving apth
CELL_CYCLE_CSV_PATH = "/home/gdallagl/myworkdir/data/MSigDB/cell_cycle_genesets.csv"

# Garated genes list
GUARANTEED_GENES_PATH = "/home/gdallagl/myworkdir/data/MSigDB/julies_cycling_signatures_cancer.tsv"

# Updated keywords pattern with word boundaries to avoid false matches
KEYWORDS_PATTERN = "|".join([
    "PROLIFER",
    "_CYCLING",  # avoid "recycling"
    "CELL_CYCLE",
    "_CC_", "_G1_", "_S_PHASE_", "_G2_", "_M_PHASE_", # avoid "aCCumbens"
    "MITOSIS", "MITOTIC",
    "CDK",
    "CHECKPOINT"
])

# Exclusion pattern
EXCLUSION_PATTERN = r"MEIOTIC|MEIOSIS|FATTY_ACID_CYCLING_MODEL"


# KEYWORDS_PATTERN = "|".join([
#     # Key regulatory phrases
#     "_CELL_CYCLE_CHECKPOINT",
#     "CELL_CYCLE_REGULATION",
#     "CELL_CYCLE_CONTROL",

#     "HALLMARK_E2F_TARGETS", 
#     " HALLMARK_G2M_CHECKPOINT",
#     # Core regulators
#     "CDK", "_CYCLIN",

#     # Transitions
#     "_G1_S", "G1_S_", "G2_M", "_S_PHASE_", "M_PHASE", "MITOTIC_CHECKPOINT"
# ])
# # Exclusion pattern
# EXCLUSION_PATTERN = r"MEIOTIC|MEIOSIS|FATTY_ACID_CYCLING_MODEL|CDK5|CANCER|APOPTOSIS|RECYCLING|SPINDLE|KINETOCHORE|CYTOKINESIS|CENTROSOME|KEGG_MEDICUS_PATHOGEN|KEGG_MEDICUS_VARIANT|KEGG_MEDICUS_REFERENCE|PREDICTED|HE_LIM_SUN_FETAL_LUNG|MALIGNANT|MORF_CDK2"

# Human proteome path
HUMAN_PROTEOME_PATH = "/home/gdallagl/myworkdir/data/UniRef50/human_proteome.tsv"

# # Uniref Apth
# UNIREF_PATH = "/home/gdallagl/myworkdir/ESMSec/data/UniRef50/uniref_identity_0_5_AND_taxonomy_id_2025_10_08.tsv" 

# mapping protein-gene apth
MAPPING_PATH = "/home/gdallagl/myworkdir/ESMSec/data/UniRef50/HUMAN_9606_idmapping.dat"

# Minimum frequency threshold for filtering ambiguous genes
MIN_FREQ_AMBIGOUS = 2

# id samplepostive class or take all postive egens
SAMPLE_POSITIVE_CLASS = False

# min number of postive samples per positive cluster
MIN_SAMPLE_N_POSITIVE = 2

# how many mroe negativ class to sampel
NEGATIVE_CLASS_MULT = 3

# choose if use as postive class only the guaranted genes
ONLY_GUARANTEED = False
only_guaranteed = "only-guaranteed_" if ONLY_GUARANTEED else ""

# savifn csv datset
FINAL_DATASET_PATH = f"/home/gdallagl/myworkdir/ESMSec/data/cell_cycle/dataset-cell-cycle_{only_guaranteed}{MIN_SAMPLE_N_POSITIVE}:{NEGATIVE_CLASS_MULT}.csv"
print(FINAL_DATASET_PATH)

# Autorelaod
%load_ext autoreload
%autoreload 2

/home/gdallagl/myworkdir/ESMSec/data/cell_cycle/dataset-cell-cycle_2:3.csv
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Select pathways related to cell cycle

Read the MSigDB and select all pathways with keyword related to field of interest.

In [3]:
### 1) Transform jsons into df
df_genesets = dataf.load_json_folder_to_df(JSON_DIR)
df_genesets = df_genesets.drop_duplicates(subset=['set_name'], keep='first')
display(df_genesets.head(2)); print(df_genesets.shape)

,set_name,collection,systematicName,pmid,exactSource,externalDetailsURL,msigdbURL,geneSymbols,filteredBySimilarity,externalNamesForSimilarTerms,source_file
0,MIR153_5P,C3:MIR:MIRDB,M30412,31504780,,http://mirdb.org/cgi-bin/mature_mir.cgi?name=h...,https://www.gsea-msigdb.org/gsea/msigdb/human/...,"[A1CF, AAK1, AASDHPPT, ABCE1, ABHD2, ABI2, ACB...",[],[],c3.mir.mirdb.v2025.1.Hs.json
1,MIR8485,C3:MIR:MIRDB,M30413,31504780,,http://mirdb.org/cgi-bin/mature_mir.cgi?name=h...,https://www.gsea-msigdb.org/gsea/msigdb/human/...,"[AAK1, ABHD18, ABL2, ABLIM1, ACVR1, ACVR2B, AC...",[],[],c3.mir.mirdb.v2025.1.Hs.json


(35134, 11)


In [4]:
### 2) Select only geneset related to interested fiedl
mask_include = df_genesets["set_name"].str.contains(KEYWORDS_PATTERN, case=False, regex=True)
mask_exclude = df_genesets["set_name"].str.contains(EXCLUSION_PATTERN, case=False, regex=True)
genesets_cell_cycle = df_genesets[mask_include & ~mask_exclude].copy()

# save
genesets_cell_cycle.to_csv(CELL_CYCLE_CSV_PATH)

print(genesets_cell_cycle.shape)
list(genesets_cell_cycle['set_name'].sort_values())

(466, 11)


['ACOSTA_PROLIFERATION_INDEPENDENT_MYC_TARGETS_DN',
 'ACOSTA_PROLIFERATION_INDEPENDENT_MYC_TARGETS_UP',
 'BENPORATH_CYCLING_GENES',
 'BENPORATH_PROLIFERATION',
 'BIOCARTA_CDK5_PATHWAY',
 'BIOCARTA_G1_PATHWAY',
 'BIOCARTA_G2_PATHWAY',
 'BOYAULT_LIVER_CANCER_SUBCLASS_G1_DN',
 'BOYAULT_LIVER_CANCER_SUBCLASS_G1_UP',
 'BUSSLINGER_ESOPHAGEAL_PROLIFERATING_BASAL_CELLS',
 'CHIANG_LIVER_CANCER_SUBCLASS_PROLIFERATION_DN',
 'CHIANG_LIVER_CANCER_SUBCLASS_PROLIFERATION_UP',
 'DI_MARTINO_MATRISOME_HIGHLY_PROLIFERATIVE_HNSCC',
 'DI_MARTINO_MATRISOME_HIGHLY_PROLIFERATIVE_HNSCC_TUMOR_CELL_DERIVED',
 'DONATO_CELL_CYCLE_TRETINOIN',
 'EGUCHI_CELL_CYCLE_RB1_TARGETS',
 'FIRESTEIN_CTNNB1_PATHWAY_AND_PROLIFERATION',
 'FIRESTEIN_PROLIFERATION',
 'FISCHER_G1_S_CELL_CYCLE',
 'FISCHER_G2_M_CELL_CYCLE',
 'FUJIWARA_PARK2_HEPATOCYTE_PROLIFERATION_DN',
 'FUJIWARA_PARK2_HEPATOCYTE_PROLIFERATION_UP',
 'GAVISH_3CA_MALIGNANT_METAPROGRAM_2_CELL_CYCLE_G1_S',
 'GAVISH_3CA_METAPROGRAM_B_CELLS_CELL_CYCLE',
 'GAVISH_3CA_METAPR

## Count in how many genesets each gene is present

Needed for later to calculate probability for sampling.

In [15]:
gene_counts_df = dataf.gene_set_counts(genesets_cell_cycle)

# if only_guaranteed in ture takeonly one gene of the one extarcted form MSigDB
if ONLY_GUARANTEED:
    gene_counts_df = gene_counts_df.iloc[[0], :]

display(gene_counts_df)
print(gene_counts_df.gene.to_list())

,gene,geneset_count
0,CDK1,112
1,CCNB1,100
2,E2F1,88
3,CDKN1A,85
4,BIRC5,84
...,...,...
9450,SANBR,1
9451,RTN4R,1
9452,RNF144B,1
9453,LAT,1


['CDK1', 'CCNB1', 'E2F1', 'CDKN1A', 'BIRC5', 'RB1', 'PLK1', 'TP53', 'AURKB', 'CCND1', 'CDK2', 'CDC20', 'CDC6', 'MAD2L1', 'CCNE1', 'UBE2C', 'CDCA8', 'NDC80', 'CENPF', 'CCNA2', 'TGFB1', 'CDK4', 'CDKN1B', 'MAD1L1', 'CHEK1', 'BUB1', 'AURKA', 'CCNE2', 'ZWINT', 'FBXO5', 'INCENP', 'CCNB2', 'SKA1', 'CHEK2', 'ATM', 'MYC', 'RAD21', 'CDC25C', 'TFDP1', 'MIR222', 'CDKN2A', 'RRM2', 'NUSAP1', 'NUF2', 'E2F3', 'SKA3', 'DTL', 'BUB1B', 'FGF10', 'TTK', 'FZR1', 'CENPE', 'IGF1', 'WEE1', 'BRCA1', 'NEK2', 'KNL1', 'ESPL1', 'CTNNB1', 'RGCC', 'CLSPN', 'BRCA2', 'BMP4', 'CDC23', 'BUB3', 'CDK6', 'CDC7', 'CCND3', 'CDC16', 'TPR', 'FGF2', 'PKMYT1', 'ORC1', 'CDC25A', 'E2F2', 'CDC25B', 'DLGAP5', 'EZH2', 'ANAPC7', 'ANAPC11', 'MDM2', 'CDCA5', 'TPX2', 'MIR21', 'TOPBP1', 'PCNA', 'SPC25', 'CCND2', 'BARD1', 'ANAPC5', 'KIF14', 'SHH', 'SPDL1', 'CDK5RAP2', 'RRM1', 'KIF20B', 'FGFR2', 'RPA2', 'PRC1', 'NF1', 'KIF23', 'RACGAP1', 'KNTC1', 'CENPJ', 'MAD2L1BP', 'ATR', 'MCM2', 'ASPM', 'ANAPC15', 'CDC45', 'MIR221', 'BLM', 'AKT1', 'FGFR1'

## Mark as "ambiguos" genes with too few Gene Sets

In [16]:
# give a label to gene that overcome the thr
gene_counts_df["label"] = gene_counts_df.geneset_count.apply(lambda x: 'positive' if x > MIN_FREQ_AMBIGOUS else 'ambigous')

# create a label for later
gene_counts_df["is_guaranteed"] = False

display(gene_counts_df)
print(gene_counts_df.label.value_counts())

,gene,geneset_count,label,is_guaranteed
0,CDK1,112,positive,False
1,CCNB1,100,positive,False
2,E2F1,88,positive,False
3,CDKN1A,85,positive,False
4,BIRC5,84,positive,False
...,...,...,...,...
9450,SANBR,1,ambigous,False
9451,RTN4R,1,ambigous,False
9452,RNF144B,1,ambigous,False
9453,LAT,1,ambigous,False


label
ambigous    5531
positive    3924
Name: count, dtype: int64


## Add Guaranteed genes

Add genes related to interesting field (i.e. that msut be present).

Read them from csv file.

Add them with max-freq.

In [17]:
### 1) Read spefic csv
garanted_genes_df = pd.read_csv(GUARANTEED_GENES_PATH, sep='\t')
display(garanted_genes_df.head(5))

### 2) Extarct the single gene names
all_values = garanted_genes_df.to_numpy().flatten().tolist()
all_values = [x for x in all_values if pd.notna(x)] # remove nan
all_values = list(set(all_values)) # remove duplicated
print("Number of guaranted genes: ", len(all_values))

### 3) Add them into previus df
# create a DataFrame for new genes
new_genes_df = pd.DataFrame({
    'gene': all_values,
    'geneset_count': max(gene_counts_df.geneset_count), # Use as Freq the max (as these genes are guaranted)
    'label': "positive",
    "is_guaranteed": True
})

# append to existing gene_frequency_df
gene_frequency_df = pd.concat([gene_counts_df, new_genes_df], ignore_index=True)

# ATTENTION: Drop duplicates, keeping **the last occurrence** (i.e., from new_genes_df) --< so gurated genes have max freq
gene_frequency_df = gene_frequency_df.drop_duplicates(subset='gene', keep='last')

# Sort and reset index
gene_frequency_df.sort_values(by=['geneset_count', 'gene'], ascending=[False, True], inplace=True)
gene_frequency_df.reset_index(drop=True, inplace=True)

display(display(gene_frequency_df.head(5)))


,GBM_G1S,GBM_G2M,H3_K27M_CC,IDH_O_G1S,IDH_O_G2M,Melanoma_G1S,Melanoma_G2M
0,RRM2,CCNB1,UBE2T,MCM5,HMGB2,MCM5,HMGB2
1,PCNA,CDC20,HMGB2,PCNA,CDK1,PCNA,CDK1
2,KIAA0101,CCNB2,TYMS,TYMS,NUSAP1,TYMS,NUSAP1
3,HIST1H4C,PLK1,MAD2L1,FEN1,UBE2C,FEN1,UBE2C
4,MLF1IP,CCNA2,CDK1,MCM2,BIRC5,MCM2,BIRC5


Number of guaranted genes:  146


,gene,geneset_count,label,is_guaranteed
0,ANLN,112,positive,True
1,ANP32E,112,positive,True
2,ARHGAP11A,112,positive,True
3,ARL6IP1,112,positive,True
4,ASF1B,112,positive,True


None

In [18]:
# create lists of genes
ambiguos_genes = set(gene_frequency_df[gene_frequency_df.label == "ambigous"].gene)
positive_genes = set(gene_frequency_df[gene_frequency_df.label == "positive"].gene)

print(gene_frequency_df.is_guaranteed.value_counts())
print("\nTotal genes of cell cycle program:", gene_frequency_df.gene.nunique())

is_guaranteed
False    9315
True      146
Name: count, dtype: int64

Total genes of cell cycle program: 9461


# Create mapping positive-gene --> geneset-freq

In [19]:
# make postive gene-freq mapping
# Filter only positive genes
positive_genes_df = gene_frequency_df[gene_frequency_df["label"] == "positive"]
# Create a mapping: gene -> frequency
positive_gene_freq_map = dict(zip(positive_genes_df["gene"], positive_genes_df["geneset_count"]))
positive_gene_freq_map

{'ANLN': 112,
 'ANP32E': 112,
 'ARHGAP11A': 112,
 'ARL6IP1': 112,
 'ASF1B': 112,
 'ATAD2': 112,
 'AURKA': 112,
 'AURKB': 112,
 'BIRC5': 112,
 'BLM': 112,
 'BRIP1': 112,
 'BUB1': 112,
 'BUB1B': 112,
 'CASP8AP2': 112,
 'CBX5': 112,
 'CCNA2': 112,
 'CCNB1': 112,
 'CCNB2': 112,
 'CCNE2': 112,
 'CDC20': 112,
 'CDC25B': 112,
 'CDC25C': 112,
 'CDC45': 112,
 'CDC6': 112,
 'CDCA2': 112,
 'CDCA3': 112,
 'CDCA5': 112,
 'CDCA7': 112,
 'CDCA8': 112,
 'CDK1': 112,
 'CENPA': 112,
 'CENPE': 112,
 'CENPF': 112,
 'CENPK': 112,
 'CENPM': 112,
 'CHAF1B': 112,
 'CKAP2': 112,
 'CKAP2L': 112,
 'CKAP5': 112,
 'CKS1B': 112,
 'CKS2': 112,
 'CLSPN': 112,
 'CTCF': 112,
 'DLGAP5': 112,
 'DSCC1': 112,
 'DTL': 112,
 'DUT': 112,
 'E2F8': 112,
 'ECT2': 112,
 'EXO1': 112,
 'FAM64A': 112,
 'FANCI': 112,
 'FEN1': 112,
 'FOXM1': 112,
 'G2E3': 112,
 'GAS2L3': 112,
 'GINS2': 112,
 'GMNN': 112,
 'GPSM2': 112,
 'GTSE1': 112,
 'H2AFZ': 112,
 'HELLS': 112,
 'HIST1H4C': 112,
 'HJURP': 112,
 'HMGB2': 112,
 'HMGB3': 112,
 'HMGN2':

# Portein --> Gene mapping

Focus only on:
- human
- reviwed
- no isoforms
- In case a protein is mapped to differt genes, take only 1 gene
    - the selected gene is choosen randomly within the possibilities
    - BUT genes that are in "positve_genes" are favored

In [20]:
proteome_uniprot = pd.read_csv(HUMAN_PROTEOME_PATH, sep="\t")

# ONLY Human 
    # csv file alredy fitered by species

# ONLY rswissprot
proteome_uniprot_reviewed = proteome_uniprot[proteome_uniprot.Reviewed == "reviewed"].copy()
print("reviewed shape: ", proteome_uniprot_reviewed.shape)

# No isofrom, use canonical protein
proteome_uniprot_reviewed['IsIsoform'] = proteome_uniprot_reviewed['Entry'].str.contains(r'-\d+$')
n_isoforms = proteome_uniprot_reviewed['IsIsoform'].sum()
n_total = len(proteome_uniprot_reviewed)
print(f"Isoforms: {n_isoforms} / {n_total} ({n_isoforms/n_total:.2%})")

reviewed shape:  (20405, 7)
Isoforms: 0 / 20405 (0.00%)


In [34]:
# ATTENTION: same protein can have mutliple genes
print("Gens mapping to Q9P1J3 -->", proteome_uniprot[proteome_uniprot["Entry"] == "Q9P1J3"]["Gene Names"].values)

# ATTENTION: soem proteins can have Nan geens
display(proteome_uniprot[proteome_uniprot["Entry"] == "Q9N2K0"])


Gens mapping to Q9P1J3 --> ['DHRS4-AS1 C14orf167 PRO1488']


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Sequence
12682,Q9N2K0,reviewed,ENH1_HUMAN,HERV-H_2q24.3 provirus ancestral Env polyprote...,NaN,Homo sapiens (Human),MIFAGKAPSNTSTLMKFYSLLLYSLLFSFPFLCHPLPLPSYLHHTI...


In [36]:
# protein to gene mapping
    # ATTENTION:
        # Mmutlipe mapping --> take first geen name
        # but tbe sure to not lsoe "positve genes"
    # ATTNETION
        # ignor prteins wiht no gene mapping

protein_to_gene_map = {}

for _, row in proteome_uniprot_reviewed.iterrows():
    entry = row['Entry']
    genes = row['Gene Names']
    
    if pd.isna(genes):
        #print(entry, genes)
        continue  # skip if no gene
    
    # split multiple genes into a list
    gene_list = genes.split()
    
    # pick one gene
    gene_selected = None
    for g in gene_list:
        if g in positive_genes:
            gene_selected = g
            break
    if gene_selected is None:
        gene_selected = gene_list[0]  # fallback to first gene
    
    protein_to_gene_map[entry] = gene_selected

# Example check
print(list(protein_to_gene_map.items())[:10])
print("Number of unique proteins in proteome:", len(set(proteome_uniprot_reviewed.Entry.unique())))
print("Number of keys in protein_to_gene_map:", len(protein_to_gene_map))
missing_proteins = set(proteome_uniprot_reviewed.Entry.unique()) - set(protein_to_gene_map.keys())
print("Protein with missing gene",  len(missing_proteins), missing_proteins)

[('A0A087X1C5', 'CYP2D7'), ('A0A0B4J2F0', 'PIGBOS1'), ('A0A0C5B5G6', 'MT-RNR1'), ('A0A0K2S4Q6', 'CD300H'), ('A0A0U1RRE5', 'NBDY'), ('A0A1B0GTW7', 'CIROP'), ('A0AV02', 'SLC12A8'), ('A0AV96', 'RBM47'), ('A0AVF1', 'IFT56'), ('A0AVI4', 'TMEM129')]
Number of unique proteins in proteome: 20405
Number of keys in protein_to_gene_map: 20293
Protein with missing gene 112 {'Q6ZRN7', 'Q9N2J8', 'Q6ZQT0', 'A8MWP4', 'Q6ZSR9', 'A8MT66', 'Q9HAA7', 'C0HM83', 'Q5PR19', 'Q8IYB0', 'Q5VSD8', 'A4D1N5', 'A8MVJ9', 'Q8N1X5', 'Q8N6K4', 'A6NL46', 'Q8NA96', 'Q8N2B8', 'A6NNC1', 'Q8TCH9', 'Q9Y3F1', 'A8MZ25', 'Q6P435', 'Q6AWC8', 'Q8N9G6', 'Q8N9W7', 'Q6ZRU5', 'Q8N8P6', 'O00370', 'Q6ZS49', 'Q6ZSR3', 'Q6ZRP5', 'A8MUA0', 'Q6ZVQ6', 'Q8N9P0', 'Q8IZM0', 'A6NJR5', 'Q56UQ5', 'Q1T7F1', 'Q6ZTI0', 'Q6ZR03', 'Q6ZN92', 'Q8NAQ8', 'Q3C1V9', 'Q8N1Y9', 'Q6ZQY7', 'A8MX80', 'Q8N377', 'Q6ZVH6', 'Q96NJ1', 'Q6ZSN1', 'Q6ZRM9', 'Q9UFV3', 'A8MV72', 'A8MTW9', 'P0C880', 'Q6Q795', 'Q8N976', 'Q6ZTK2', 'A8MUU9', 'Q8N3U1', 'Q9BTK2', 'Q6ZPA2', 'Q6ZR

In [37]:
# define allowed prteisn to use in furtyehr analays
print("Proteins allowed in this analysis: ", len(protein_to_gene_map))
allowed_proteins = list(protein_to_gene_map.keys())

Proteins allowed in this analysis:  20293


In [ ]:
# check for dupliuctaed genes or proteins

# convert the mapping to a DataFrame
mapping_df = pd.DataFrame(list(protein_to_gene_map.items()), columns=["protein", "gene"])

# dupalcated proteins
print("number duplciated proteins: ", mapping_df.protein.duplicated().sum()) # no protesin mapped to multiple genes
print("number duplciated genes: ", mapping_df.gene.duplicated().sum())

# gene to whcih rpoteins maps
mapping_df.groupby("gene")["protein"].apply(list).loc[lambda x: x.str.len() > 1].head()



number duplciated proteins:  0
number duplciated genes:  143


gene
ACAT2            [O75908, Q9BWD1]
AK6              [Q9UIJ7, Q9Y3D8]
AKAP7            [O43687, Q9P0M2]
APPL2            [Q06481, Q8NEU8]
ATR      [P20848, Q13535, Q9H6X2]
Name: protein, dtype: object

# Create Uniref50 clusters


In [ ]:
uniprot_id_mapping_df = pd.read_csv(MAPPING_PATH, sep='\t', header=None, names=['UniProtKB_Accession', 'ID_Type', 'External_ID'])
#print(uniprot_id_mapping_df.ID_Type.unique())

# select protein IDs
protein2uniref = uniprot_id_mapping_df[uniprot_id_mapping_df.ID_Type == "UniRef50"]
print(protein2uniref.shape)

# FIlter only  (--> using lsit og before)
    # HUMANS 
    # REVIEWED
    # NO ISOFORMS
protein2uniref = protein2uniref[protein2uniref.UniProtKB_Accession.isin(all_uniprot_reviewed_proteins)]
print(protein2uniref.shape)

# reanme
protein2uniref.drop(columns=["ID_Type"], inplace=True)
protein2uniref.rename(columns={
    "External_ID": "UniRef50_Cluster",
    "UniProtKB_Accession": "protein"
}, inplace=True)
display(protein2uniref)

# filter by ALLOWED proteins
    # probabily losesome proteins
protein2uniref = protein2uniref[protein2uniref.protein.isin(allowed_proteins)]
print(protein2uniref.shape)
print("number unique clusters: ", protein2uniref.UniRef50_Cluster.nunique())


In [ ]:
# create a df with clusters as entry
uniref_df = protein2uniref.groupby('UniRef50_Cluster').agg({
    'protein': list,
}).reset_index()

# add gene name
uniref_df["gene"] = uniref_df["protein"].apply(
    lambda lst: [protein_to_gene_map.get(p, None) for p in lst]
)

# count
uniref_df["n_proteins"] = uniref_df.protein.apply(len)
uniref_df["n_genes"] = uniref_df.gene.apply(len)
uniref_df.sort_values(by="n_proteins", inplace=True, ascending=False)

# plot
display(uniref_df)
uniref_df.n_proteins.value_counts().sort_index().plot(kind="bar", logy=True)

In [ ]:
# genes or proteins shared acorss clusters

# explode the lists into one row per (cluster, protein)
protein_cluster_map = uniref_df.explode("protein")[["UniRef50_Cluster", "protein"]].dropna()
gene_cluster_map = uniref_df.explode("gene")[["UniRef50_Cluster", "gene"]].dropna()

# group by protein and collect all clusters that contain it
protein_to_clusters = protein_cluster_map.groupby("protein")["UniRef50_Cluster"].unique()
gene_to_clusters = gene_cluster_map.groupby("gene")["UniRef50_Cluster"].unique()

# keep only proteins that occur in >1 cluster
shared_proteins = protein_to_clusters[protein_to_clusters.apply(len) > 1]
shared_genes = gene_to_clusters[gene_to_clusters.apply(len) > 1]

print(f"{len(shared_proteins)} proteins are shared between clusters")
display(shared_proteins.head())
print(f"{len(shared_genes)} genes are shared between clusters")
display(shared_genes.head())


In [ ]:
uniref_df[uniref_df["gene"].apply(lambda genes: "ATR" in genes if isinstance(genes, list) else False)]


## Mark positive genes and clusters

In [ ]:
# Add the 'label' column: 'positive' if at least one protein maps to a positive gene, otherwise 'negative'
uniref_df["cluster_label"] = uniref_df["gene"].apply(
    lambda gene_list: "positive" 
    if any(g in positive_genes for g in gene_list)
    else "negative"
)

# plost how many postive genesets
print(uniref_df["cluster_label"].value_counts())#.sort_index().plot(kind="bar", logy=True,)

#########

# create list of genes AND CORREPSONDET prots that are postive (from above)

# unzip in parallel pritens_clened and genes_clened and create ritens_clened_psotive and genes_clened_positive 
    # maintain if proteins comes form psotive gene
uniref_df["proteins_positive"], uniref_df["genes_positive"] = zip(*uniref_df.apply(
    lambda row: (
        [p for p, g in zip(row["protein"], row["gene"]) if g in positive_genes],
        [g for g in row["gene"] if g in positive_genes]
    ),
    axis=1
))
uniref_df["n_genes_positive"] = uniref_df["genes_positive"].apply(len)
uniref_df.sort_values(by="n_genes_positive", ascending=False, inplace=True)

display(uniref_df.head(5))
display(uniref_df.shape)
uniref_df.n_genes_positive.value_counts().plot(kind="bar", logy=True)

In [ ]:
# give probablities to be samped to each protein
uniref_df["logits"] = uniref_df["genes_positive"].apply(
    lambda gene_list: [positive_gene_freq_map.get(g, 0) for g in gene_list]
)
def safe_softmax(logits):
    if len(logits) == 0:
        return []  # return empty list if no logits
    return softmax(logits).tolist()  # convert numpy array to list
uniref_df["probs"] = uniref_df["logits"].apply(safe_softmax)

uniref_df.head(2)

In [ ]:
# Create list of putative negative genes/protein        
    # putatove = neither postive neither ambigous
gene_to_exclude = ambiguos_genes.union(positive_genes)

uniref_df["putative_negative_proteins"], uniref_df["putative_negative_genes"] = zip(*uniref_df.apply(
    lambda row: (
        [p for p, g in zip(row["protein"], row["gene"]) if g not in gene_to_exclude],
        [g for g in row["gene"] if g not in gene_to_exclude]
    ),
    axis=1
))

uniref_df["n_putative_negative_genes"] = uniref_df.putative_negative_genes.apply(len)

display(uniref_df.tail(5))


## Plotting positive genes per cluster

In [ ]:
# FRACTION OF CLUSTER OCCUPIED BY POSITIVE GENES
uniref_df["fraction_positive"] = uniref_df.n_genes_positive / uniref_df.n_genes
uniref_df.sort_values(by="fraction_positive", inplace=True, ascending=False)

# hsit
uniref_df.fraction_positive.plot(kind="hist", logy=True, bins=50)
print(uniref_df.fraction_positive.value_counts().sort_index())
uniref_df

In [ ]:
import pandas as pd
from scipy.stats import hypergeom, binom
import numpy as np

uniref_df_big_clusters = uniref_df[uniref_df.n_genes >= 5].copy()


# Calculate total genes and total positive genes across all clusters
total_genes = uniref_df_big_clusters['n_genes'].sum()
total_positive = uniref_df_big_clusters['n_genes_positive'].sum()

# Hypergeometric Test for each cluster
def hypergeom_test(cluster_row, total_genes, total_positive):
    """
    Tests if cluster is enriched for positive genes
    
    Parameters:
    - cluster_row: row from dataframe with n_genes and n_genes_positive
    - total_genes: total number of genes across all clusters
    - total_positive: total number of positive genes across all clusters
    """
    # Hypergeometric parameters:
    # M = total population size
    # n = number of success states in population (positive genes)
    # N = number of draws (genes in cluster)
    
    M = total_genes
    n = total_positive
    N = cluster_row['n_genes']
    k = cluster_row['n_genes_positive']
    
    # One-tailed p-value (testing if cluster has MORE positive genes than expected)
    p_value = 1 - hypergeom.cdf(k - 1, M, n, N)
    
    # Calculate probability of observing exactly k positive genes
    prob = hypergeom.pmf(k, M, n, N)
    
    return prob, p_value

# Apply to each cluster - get both prob and pvalue in one go
results = uniref_df_big_clusters.apply(
    lambda row: pd.Series(hypergeom_test(row, total_genes, total_positive)), 
    axis=1
)
uniref_df_big_clusters['enrichment_prob'] = results[0]
uniref_df_big_clusters['enrichment_pvalue'] = results[1]

# Sort by p-value to see most significant clusters
uniref_df_results = uniref_df_big_clusters.sort_values('enrichment_pvalue')

print("Top enriched clusters:")
display(uniref_df_results[['n_genes', 'n_genes_positive', 'fraction_positive', 
                          'enrichment_prob', 'enrichment_pvalue']])

# Apply multiple testing correction (Bonferroni)
n_tests = len(uniref_df_big_clusters)
alpha = 0.05
bonferroni_threshold = alpha / n_tests

uniref_df_big_clusters['significant'] = uniref_df_big_clusters['enrichment_pvalue'] < bonferroni_threshold

print(f"\nNumber of significantly enriched clusters (Bonferroni corrected): {uniref_df_big_clusters['significant'].sum()}")
print(f"Bonferroni threshold: {bonferroni_threshold:.2e}")


## Positive class sampling


In [ ]:
def sample_sampled_from_single_row(row, min_sample_n, prot_col, gene_col, probs_col):

    N = min(len(row[prot_col]), min_sample_n)

    if probs_col != None:
        probs = row[probs_col] #use precompued prbs
    else:
        probs=None # unirform prob

    # sample indices
    sampled_indices = np.random.choice(len(row[prot_col]), size=N, replace=False, p=probs) 

    # covert to array
    proteins_array = np.array(row[prot_col])
    genes_array = np.array(row[gene_col])

    # return sliced
    return proteins_array[sampled_indices], genes_array[sampled_indices]

In [ ]:
# Filter only positive clusters
uniref_df_pos = uniref_df[uniref_df.cluster_label == "positive"].copy()

# Take ALL psotive (not sample)
uniref_df_pos["proteins_sampled"] = uniref_df_pos["proteins_positive"]
uniref_df_pos["genes_sampled"] =  uniref_df_pos["genes_positive"]
uniref_df_pos["n_proteins_sampled"] = uniref_df_pos.proteins_sampled.apply(len)

uniref_df_pos

## Negative class sampling

In [ ]:
# Filter only positive clusters
    # sample from ONLY NEGATIVE --> no over,ap
    # but they shiudl have at least 1 neagtive protein
uniref_df_neg = uniref_df[(uniref_df.n_putative_negative_genes > 0) & (uniref_df.cluster_label == "negative")].copy() 

uniref_df_neg["proteins_sampled"], uniref_df_neg["genes_sampled"] = zip(
    *uniref_df_neg.apply(
            sample_sampled_from_single_row, 
            axis=1, 
            min_sample_n=MIN_SAMPLE_N_POSITIVE, 
            gene_col="putative_negative_genes", 
            prot_col="putative_negative_proteins", 
            probs_col=None # uniform
            )
)
uniref_df_neg["n_proteins_sampled"] = uniref_df_neg.proteins_sampled.apply(len)

# ATTENTION: same clusters coudl have 0 proteins to sample from (all proteins are postive OR ambigous so nothing left to sample as negative)
uniref_df_neg = uniref_df_neg[uniref_df_neg.n_proteins_sampled > 0]

display(uniref_df_neg.head())
print(uniref_df_neg.shape)

## Make dataset

Final df shoudl be:
- | Cluster | protein name | label | seq |

In [ ]:
# add labels
uniref_df_pos["label_single_prot"] = 1
uniref_df_neg["label_single_prot"] = 0

# merge --> each entry is a cluster
dataset_df = pd.concat([uniref_df_pos, uniref_df_neg])
display(dataset_df.head())


In [ ]:
# smaller df
cols_to_keep = ['UniRef50_Cluster', 'proteins_sampled',
                'genes_sampled', 'label_single_prot']
dataset_df_small = dataset_df[cols_to_keep].copy()

display(dataset_df_small)
print("ATTENTION: the number of proteins sampled is less than the total ~20000 of Swissport, as we subsampled only N, so we may have lsot some proteins")
print(len(set(gene for sublist in dataset_df_small['proteins_sampled'] for gene in sublist)), "/ 20000")

# Create a new DataFrame where each row corresponds to a (protein, gene) pair
rows = []
for _, row in dataset_df_small.iterrows():
    cluster = row["UniRef50_Cluster"]
    label = row["label_single_prot"]
    proteins = row["proteins_sampled"]
    genes = row["genes_sampled"]

    # zip ensures 1-to-1 pairing between proteins and genes
    for prot, gene in zip(proteins, genes):
        rows.append({
            "UniRef50_Cluster": cluster,
            "protein": prot,
            "gene": gene,
            "label": label
        })

df_long = pd.DataFrame(rows)

# checj fro duplcates
print("\nDuplicated rows/proteins: ", df_long.shape[0] - df_long.protein.nunique())

df_long

# Assure 1:3 ratio in postive and negative class
After this sampling you will have (done LATER in notebook)
- | pos prot | << | neg prot | → more thna 1:3 ration
- Need to subsampled negative class
- Subsample negative class
    - Retain all proteins that comes form clusters that have a positive protein
    - Subsample the remaining cluster to have a final ratio 1:3

In [ ]:
# list of smpled positve proteins
positive_proteins_tmp = set(df_long[df_long.label == 1]["protein"].unique())

# list of prtein in MIXED cluster (wiht inside btoh postive and negative prteins)
    # ATTENTUON: check no ptortien in mixed clusters
proteins_in_mixed_clusters = df_long.groupby('UniRef50_Cluster').filter(lambda x: x['label'].nunique() > 1).sort_values(by="UniRef50_Cluster")
print("priteins in mixed clusters:", len(proteins_in_mixed_clusters))
proteins_in_mixed_clusters = set(proteins_in_mixed_clusters["protein"].unique())

# proteins to NOT remove
proteins_not_to_remove = proteins_in_mixed_clusters.union(positive_proteins_tmp)

# give col
df_long["can_be_removed"] = ~df_long["protein"].isin(proteins_not_to_remove)

df_long

In [ ]:
# decide fraction to susample

# Number of positive proteins
n_positive = df_long[df_long["label"] == 1].shape[0]
# Number desidered negative
n_desired_negatives = n_positive * NEGATIVE_CLASS_MULT
# Removable (negative) proteins
removable_negatives = df_long[(df_long["label"] == 0) & (df_long["can_be_removed"])]
# Determine fraction to subsample
frac_to_sample = n_desired_negatives / len(removable_negatives)
frac_to_sample = min(frac_to_sample, 1.0)  # cannot sample more than available

frac_to_sample

In [ ]:

# susample
removable = df_long[df_long["can_be_removed"]]
keep = df_long[~df_long["can_be_removed"]]

subsampled_removable = removable.sample(frac=frac_to_sample, random_state=SEED)

df_subsampled = pd.concat([keep, subsampled_removable]).reset_index(drop=True) # Combine back with the protected proteins

print(df_subsampled.label.value_counts())
df_subsampled

## Give Seq


In [ ]:
### create df to join
join_df_tmp = proteome_uniprot[["Entry", "Sequence"]].copy().rename(columns={"Sequence": "sequence"})

# join
    # ATTNETION: we shoudl NOT lose any col
long_df_seq = pd.merge(how="inner", left=df_subsampled, right=join_df_tmp, left_on="protein", right_on="Entry")

# drp cols
long_df_seq.drop(columns=["can_be_removed", "Entry"], inplace=True)

long_df_seq

## Stratify dataset

We can stratify based on many metrics:
1. labels
2. PREVALENT metric
3. **based on cluster**
    - Put proteins coming from same cluster in different sets or not

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# --- group-level stratified split ---

# pslit by cluster and then ive back set-label to each row

# One representative row per UniRef50_Cluster
cluster_df = long_df_seq.groupby('UniRef50_Cluster').first().reset_index()

# Stratified split by label, but at the cluster level
    # ists of unique cluster IDs.
train_clusters, temp_clusters = train_test_split(
    cluster_df['UniRef50_Cluster'],
    test_size=0.2,
    stratify=cluster_df['label'],
    random_state=42
)

# Split the temporary set equally into validation and test
temp_labels = cluster_df.loc[cluster_df['UniRef50_Cluster'].isin(temp_clusters), 'label']
val_clusters, test_clusters = train_test_split(
    temp_clusters,
    test_size=0.5,
    stratify=temp_labels,
    random_state=42
)

# --- assign back to the full dataframe ---
#Every row in long_df_seq inherits its set label based on its UniRef50_Cluster
long_df_seq['set'] = ''
long_df_seq.loc[long_df_seq['UniRef50_Cluster'].isin(train_clusters), 'set'] = 'train'
long_df_seq.loc[long_df_seq['UniRef50_Cluster'].isin(val_clusters), 'set'] = 'val'
long_df_seq.loc[long_df_seq['UniRef50_Cluster'].isin(test_clusters), 'set'] = 'test'

# Check split distribution
print(long_df_seq.groupby('set')['label'].value_counts().unstack(fill_value=0))
display(long_df_seq)


# Optional: add precomputed protein embs

In [ ]:
#long_df_seq["embedding"] = [[] for _ in range(len(long_df_seq))]

## Save

In [ ]:
long_df_seq.to_csv(FINAL_DATASET_PATH)
FINAL_DATASET_PATH

# ------------------------------

In [ ]:
from Bio import SeqIO
import pandas as pd

# Parse all sequences in the FASTA file
records = list(SeqIO.parse("/home/gdallagl/myworkdir/ESMSec/data/cell_cycle/0_class_yu_et_all.fasta", "fasta"))
data = {
    "protein": [record.id for record in records],
    "sequence": [str(record.seq) for record in records],
    "label": [0 for record in records],
}
df_neg = pd.DataFrame(data)
print(df_neg.shape)


records = list(SeqIO.parse("/home/gdallagl/myworkdir/ESMSec/data/cell_cycle/1_class_yu_et_all.fasta", "fasta"))
data = {
    "protein": [record.id for record in records],
    "sequence": [str(record.seq) for record in records],
    "label": [1 for record in records],
}
df_pos = pd.DataFrame(data)
print(df_pos.shape)

df = pd.concat([df_neg, df_pos])

df


In [ ]:
train_idx, test_idx = train_test_split(
    df.index,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

# Assign splits
df['set'] = ''
df.loc[train_idx, 'set'] = 'train'
df.loc[test_idx, 'set'] = 'test'
df.loc[test_idx[0], 'set'] = 'val' #just temp
df.reset_index(drop=True, inplace=True)
df["protein"] = range(df["protein"].shape[0])

print(df.groupby('set')['label'].value_counts().unstack(fill_value=0))
display(df)
df

In [ ]:
df.iloc[[332]].sequence.to_list()

In [ ]:
df.to_csv("/home/gdallagl/myworkdir/ESMSec/data/cell_cycle/yu_et_all_cyclins.csv")